# Goals:

**1. Develop a natural language processor capable of predicting the COLOR of a magic card based on the rules text of that card.**


In [1]:
# imports and display options

import pandas as pd
import numpy as np
import math
from math import sqrt

from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import unicodedata
import re
import nltk
from nltk.corpus import stopwords

import prepare as p
# import explore as e
# import model as m

pd.set_option('display.max_colwidth', -1)

# Acquire

* Used file from previouse 
* A CSV, containing an up to date breakdown of each card that has been printed so far, was obtained from MTGJSON.com
* Each row represents a card or a version of a card
* The CSV was read into a pandas dataframe
* The original dataframe contained 51,430 rows and 73 columns

# Prepare

The following steps were taken to prepare the data:

1. Restricted dataframe to relevant columns
 
2. Restricted dataframe to rows containing cards that exist in physical form

3. Restricted dataframe to rows containing a value for 'text'

4. Restricted dataframe to rows with a single 'color identity' (see data dictionary Color)

5. Dropped all columns other than text and color

6. Renamed columns

7. Changed some of the symble represintations to words to make them machine readable

8. Applied basic cleaning to values in text by, lowercasing letters, converting to ASCII characters, removing non-letter characters, lemmatizeing the words, and removing stopwords

9. Dropped duplicate rows

10. Changed column order

11. Wrote prepared data to ‘mtgprep.csv’ for ease of access. Data consisted of 15380 rows and two columns.

12. Created a test and train group at a 20/80 split

## Data Dictionary:

* Magic: The Gathering
    * a collectable card game developed by Wizards of the Coast Inc. In a typical game, each player combines a selection of cards from their collection into his or her own deck, which they use to compete against other players. Thematically, players assume the roles of powerful mages fighting for supremacy. Cards in each players deck represent spells and other resources each player’s disposal. A game typically ends when all but one of the participating players is reduced to zero “life” or is otherwise eliminated from the game.

### Columns

* Color
    * Spells players can cast are divided into five different colors Each color is thematically distinct
    * White represents order and morality
    * Blue represents cunning and technology
    * Black represents pragmatism and amorality
    * Red represents impulse and chaos
    * Green represents nature and instinct
    * Spells with more than one color were excluded for the study to give a clearer picture of sentiment values for each color
    * The color or colors each spell had was determined by the color of mana symbols that appear on each card, also called its color identity
    <br />   
    
    
* Text
    * The value in text represents the rules text appearing on one Magic the Gathering card

In [2]:
# prepare data 
df = p.get_preped_data()

#create test and train groups
train, test = p.split_data(df)

# Explore

In [8]:
test.head(25)

,color,text
8035,Blue,reveal top five card library opponent separate card two pile put one pile hand graveyard
42753,White,destroy target creature dealt damage turn
6736,Black,exile target creature create gold token artifact sacrifice artifact add one mana color
3115,Blue,look top two card library put one hand bottom library
7589,Red,player gain life beginning upkeep witch hunt deal damage beginning end step target opponent chosen random gain control witch hunt
46063,Red,mogg conscript attack unless cast creature spell turn
19747,Blue,tap target creature opponent control creature untap controller next untap step cipher may exile spell card encoded creature control whenever creature deal combat damage player controller may cast copy encoded card without paying mana cost
18445,Black,blackmana tap sacrifice another creature manifest top card library put card onto battlefield face creature turn face time mana cost creature card
20068,Red,target blocking creature get plusandplus end turn
5084,White,whitemana target nonattacking nonblocking creature get plusandplus end turn
